In [1]:
import math
import numpy as np

In [2]:
def Estep(X, mixture_weights, means, cov_mat, p, G):
    # multivariate gaussian PDF
    probs = np.zeros((X.shape[0], G)) # n x g shape
    for k, mu in enumerate(means): # exponent term
        probs[:,k] = np.sum((np.linalg.inv(cov_mat) @ (X - mu).T) * (X - mu).T, axis=0)
    probs = np.exp(probs * -0.5)
    probs /= math.sqrt(np.linalg.det(cov_mat) * (2 * math.pi) ** p) 

    probs *= mixture_weights

    return (probs / probs.sum(axis=1, keepdims=True)) + 1e-10 # normalizing

In [3]:
def Mstep(X, probs, p, G):
    mixture_weights = np.mean(probs, axis=0)
    means = np.zeros((G, p))
    cov_mat = np.zeros((p, p))
    for k in range(G):
        means[k] = np.sum((X.T * r[:,k]).T, 0) / sum(r[:,k])
        tmp = X - means[k]
        cov_mat_k = tmp.T @ np.diag(probs[:,k]) @ tmp
        cov_mat_k /= np.sum(probs[:,k])
        cov_mat += cov_mat_k
    return mixture_weights, means.T, cov_mat

In [4]:
faithful = open('faithful.dat')
temp = []
for row in faithful:
    temp.append(row.split()[1:])
temp = temp[1:]
for i in range(len(temp)):
    for j in range(len(temp[i])):
        temp[i][j] = float(temp[i][j])
faithful = np.array(temp)

In [5]:
## Testing G = 2
n = len(faithful)
X = faithful
p = 2
G = 2

p1 = 10/n
p2 = 1 - p1
mixture_weights = [p1, p2]

mu1 = X[:10].mean(0)
mu2 = X[10:].mean(0)
means = np.array([mu1, mu2])

cov_mat_L = sum([(X[l] - mu1).reshape((2, 1)) @ (X[l] - mu1).reshape((1, 2)) for l in range(10)])
cov_mat_R = sum([(X[l] - mu1).reshape((2, 1)) @ (X[l] - mu1).reshape((1, 2)) for l in range(10, len(X))])
cov_mat = (cov_mat_L + cov_mat_R) / n

for i in range(5):
    r = Estep(X, mixture_weights, means, cov_mat, p, G)
    mixture_weights, means, cov_mat = Mstep(X, r, p, G)
    print(mixture_weights)


[0.03602766 0.96397234]
[1.e+00 1.e-10]
[1.e+00 1.e-10]
[1.e+00 1.e-10]
[1.e+00 1.e-10]


In [6]:
probs = np.zeros((X.shape[0], G)) # n x g shape
for k, mu in enumerate(means): # exponent term
    probs[:,k] = np.sum((np.linalg.inv(cov_mat) @ (X - mu).T) * (X - mu).T, axis=0)
probs = np.exp(probs * -0.5)
probs /= math.sqrt(np.linalg.det(cov_mat) * (2 * math.pi) ** p) 
print(probs)

[[6.38217135e-20 0.00000000e+00]
 [1.28691207e-17 0.00000000e+00]
 [6.05503738e-19 0.00000000e+00]
 [9.82416634e-19 0.00000000e+00]
 [3.37584137e-18 0.00000000e+00]
 [3.33173026e-13 0.00000000e+00]
 [9.14291685e-19 0.00000000e+00]
 [8.01319761e-23 0.00000000e+00]
 [1.15327637e-15 0.00000000e+00]
 [4.81067132e-19 0.00000000e+00]
 [1.86360065e-17 0.00000000e+00]
 [1.09546667e-20 0.00000000e+00]
 [1.11531586e-16 0.00000000e+00]
 [5.47885699e-15 0.00000000e+00]
 [1.31545305e-16 0.00000000e+00]
 [4.36640490e-15 0.00000000e+00]
 [1.59774451e-21 0.00000000e+00]
 [1.34366779e-16 0.00000000e+00]
 [9.70578429e-18 0.00000000e+00]
 [7.02579907e-17 0.00000000e+00]
 [2.33879312e-16 0.00000000e+00]
 [5.47885699e-15 0.00000000e+00]
 [3.32271079e-20 0.00000000e+00]
 [5.29168708e-18 0.00000000e+00]
 [9.69261166e-14 0.00000000e+00]
 [7.86825123e-22 0.00000000e+00]
 [3.10648772e-17 0.00000000e+00]
 [2.34367526e-16 0.00000000e+00]
 [2.91132285e-18 0.00000000e+00]
 [4.31941610e-16 0.00000000e+00]
 [2.879303

In [98]:
#BW
def forward(t,i,x, mz,w,A,B):
    if t<=0:#changed from 1
        return w[i]*B[i,x[0]]
    else:
        s=0
        for j in range(mz):
            s+=forward(t-1,j,x, mz,w,A,B)*A[j,i]*B[i,x[t-1]]
        return s
        

In [93]:
def backward(t,i,x, mx, mz,w,A,B):
    if t >= mx-1:#changed from mx
        return 1
    else:
        s = 0
        for j in range(mz):
            s+=A[i,j]*B[j,x[t]]*backward(t+1,j,x, mx, mz,w,A,B)
            print(s,mz)
        return s

In [74]:
def biggamma(t,i,j,x, mx,mz,w,A,B):
    return forward(t,i,x, mz,w,A,B)*A[i,j]*B[j,x[t]]*backward(t+1,j,x, mx, mz,w,A,B)

In [75]:
def mygamma(t,i,x, mx, mz,w,A,B):
    if t==0:#changed from 1
        s = 0
        for j in range(mz):
            s+=biggamma(1,i,j,x, mx,mz,w,A,B)
        return s
    elif t == len(x)-1:#changed from len(x)
        s = 0
        for j in range(mz):
            s+=biggamma(len(x)-1,i,j,x, mx,mz,w,A,B)
        return s
    else:
        s = 0
        for j in range(mz):
            s+=biggamma(t,i,j,x, mx,mz,w,A,B)
        return s

In [ ]:
def BW_onestep(data, mx, mz,w,A,B):
    #Estep 
    n = len(data)
    for i in range(mz):
        for j in range(mz):
            num = 0
            for t in range(n):
                num += biggamma(t,i,j,data, mx, mz,w,A,B)
                print("done with ",i,j,n)
            den = 0
            print("done with ",i,j)
        for jp in range(mz):
            for t in range(n):
                den+= biggamma(t,i,jp,data, mx, mz,w,A,B)
            A[i][j] = num/den
    
    for i in range(mz):
        for l in range(mx):
            num = 0
            T = len(data)
            for t in range(T):
                if x[t] == l:
                    num+= mygamma(t,i,data, mx,mz,w,A,B)
            den = 0
            for t in range(T):
                den+= mygamma(t,i,data, mx,mz,w,A,B)
            B[i][l] = num/den
    return(A,B)

In [ ]:
#test data
part2 = open('coding4_part2_data.txt')
temp = []
for row in part2:
    temp.append(row.split())
for i in range(len(temp)):
    temp[i] = int(temp[i][0])-1#changed because it was one indexed
part2 = np.array(temp)
w = np.array([.5,.5])
A = np.array([[.5,.5],
              [.5,.5]])
B = np.array([[1/9,3/9,5/9],
              [1/6,2/6,3/6]])
mz = 2
mx=3

In [ ]:
BW_onestep(part2, mx, mz,w,A,B)

In [ ]:
num=0
for t in range(len(part2)):
    num += biggamma(t,0,0,part2, mx, mz,w,A,B)
    print("done with ",t)

done with  0
done with  1
done with  2
done with  3
done with  4
done with  5
done with  6
done with  7
done with  8
done with  9
done with  10
done with  11
done with  12
done with  13
done with  14
done with  15
done with  16
done with  17
done with  18
done with  19
done with  20
done with  21
done with  22
done with  23
done with  24
done with  25
done with  26


In [84]:
a = 6
print("hi", 6)

hi 6


In [104]:
forward(27,0,part2, mz,w,A,B)

1.4910374631776894e-13

In [103]:
backward(28,0,part2, mx, mz,w,A,B)

1

In [102]:
biggamma(27,0,0,part2, mx, mz,w,A,B)

KeyboardInterrupt: 